## Data Cleaning sample and Working with Local Database connection


In [3]:
import pandas as pd
import logging
import re
import os
import emoji
from IPython.display import display  # Ensures DataFrames are shown properly in Jupyter

# Ensure logs folder exists
os.makedirs("./logs", exist_ok=True)

# Configure logging for Jupyter Notebook
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("./logs/data_cleaning.log"),  # Save logs to file
        logging.StreamHandler()  # Display logs in Jupyter Notebook
    ]
)

def load_csv(file_path):
    """ Load CSV file into a Pandas DataFrame. """
    try:
        df = pd.read_csv(file_path)
        logging.info(f"✅ CSV file '{file_path}' loaded successfully.")
        display(df.head())  # Show first few rows in Jupyter Notebook
        return df
    except Exception as e:
        logging.error(f"❌ Error loading CSV file: {e}")
        raise

def extract_emojis(text):
    """ Extract emojis from text, return 'No emoji' if none found. """
    emojis = ''.join(c for c in text if c in emoji.EMOJI_DATA)
    return emojis if emojis else "No emoji"

def remove_emojis(text):
    """ Remove emojis from the message text. """
    return ''.join(c for c in text if c not in emoji.EMOJI_DATA)

def extract_youtube_links(text):
    """ Extract YouTube links from text, return 'No YouTube link' if none found. """
    youtube_pattern = r"(https?://(?:www\.)?(?:youtube\.com|youtu\.be)/[^\s]+)"
    links = re.findall(youtube_pattern, str(text))  # Ensure text is a string
    return ', '.join(links) if links else "No YouTube link"

def remove_youtube_links(text):
    """ Remove YouTube links from the message text. """
    youtube_pattern = r"https?://(?:www\.)?(?:youtube\.com|youtu\.be)/[^\s]+"
    return re.sub(youtube_pattern, '', str(text)).strip()

def clean_text(text):
    """ Standardize text by removing newline characters and unnecessary spaces. """
    if pd.isna(text):
        return "No Message"
    return re.sub(r'\n+', ' ', str(text)).strip()

def clean_dataframe(df):
    """ Perform all cleaning and standardization steps while avoiding SettingWithCopyWarning. """
    try:
        df = df.drop_duplicates(subset=["ID"]).copy()  # Ensure a new copy
        logging.info("✅ Duplicates removed from dataset.")

        # ✅ Convert Date to datetime format
        df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df['Date'] = df['Date'].where(df['Date'].notna(), None)
        logging.info("✅ Date column formatted to datetime.")

        # ✅ Convert 'ID' to integer for PostgreSQL BIGINT compatibility
        df['ID'] = pd.to_numeric(df['ID'], errors="coerce").fillna(0).astype(int)

        # ✅ Fill missing values
        df['Message'] = df['Message'].fillna("No Message")
        df['Media Path'] = df['Media Path'].fillna("No Media")
        logging.info("✅ Missing values filled.")

        # ✅ Standardize text columns
        for col in ["Channel Title", "Channel Username", "Message", "Media Path"]:
            df[col] = df[col].astype(str).str.strip()

        df["Message"] = df["Message"].apply(clean_text)

        # ✅ Extract emojis and store them in a new column
        df["emoji_used"] = df["Message"].apply(extract_emojis)

        # ✅ Remove emojis from message text
        df["Message"] = df["Message"].apply(remove_emojis)

        # ✅ Extract and remove YouTube links
        df["youtube_links"] = df["Message"].apply(extract_youtube_links)
        df["Message"] = df["Message"].apply(remove_youtube_links)

        # ✅ Rename columns for PostgreSQL compatibility
        df = df.rename(columns={
            "Channel Title": "channel_title",
            "Channel Username": "channel_username",
            "ID": "message_id",
            "Message": "message",
            "Date": "message_date",
            "Media Path": "media_path",
            "emoji_used": "emoji_used",
            "youtube_links": "youtube_links"
        })

        logging.info("✅ Data cleaning completed successfully.")
        display(df.head())  # Show cleaned DataFrame in Jupyter Notebook
        return df
    except Exception as e:
        logging.error(f"❌ Data cleaning error: {e}")
        raise

def save_cleaned_data(df, output_path):
    """ Save cleaned data to a new CSV file. """
    try:
        df.to_csv(output_path, index=False)
        logging.info(f"✅ Cleaned data saved successfully to '{output_path}'.")
        print(f"✅ Cleaned data saved successfully to '{output_path}'.")
    except Exception as e:
        logging.error(f"❌ Error saving cleaned data: {e}")
        raise

# Run each step interactively in Jupyter Notebook
input_csv = '../merged_data.csv'
output_csv = 'cleaned_data.csv'

# Load the data
df = load_csv(input_csv)

# Clean the data
cleaned_df = clean_dataframe(df)

# Save the cleaned data
save_cleaned_data(cleaned_df, output_csv)


2025-01-31 20:16:39,568 - INFO - ✅ CSV file '../merged_data.csv' loaded successfully.


,Channel Title,Channel Username,ID,Message,Date,Media Path,source_channel
0,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15208,NEUROPRO CARE \nPrice 5500 birr \nTelegram @Lo...,2025-01-31 12:40:44+00:00,photos/@lobelia4cosmetics_15208.jpg,lobelia4cosmetics_data
1,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15207,ENSURE 850GM\nPrice 3800 birr \nTelegram https...,2025-01-31 12:40:44+00:00,photos/@lobelia4cosmetics_15207.jpg,lobelia4cosmetics_data
2,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15206,NIDO 1+ 2.2KG \nPrice 6500 birr \nTelegram @Lo...,2025-01-31 12:40:44+00:00,photos/@lobelia4cosmetics_15206.jpg,lobelia4cosmetics_data
3,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15205,Enfagrow A+\nPrice 5500 birr \nTelegram https:...,2025-01-31 12:40:44+00:00,photos/@lobelia4cosmetics_15205.jpg,lobelia4cosmetics_data
4,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15204,go & grow 850gm\nPrice 5500 birr \nTelegram ht...,2025-01-31 12:40:44+00:00,photos/@lobelia4cosmetics_15204.jpg,lobelia4cosmetics_data


2025-01-31 20:16:39,593 - INFO - ✅ Duplicates removed from dataset.
2025-01-31 20:16:39,608 - INFO - ✅ Date column formatted to datetime.
2025-01-31 20:16:39,612 - INFO - ✅ Missing values filled.
2025-01-31 20:16:39,646 - INFO - ✅ Data cleaning completed successfully.


,channel_title,channel_username,message_id,message,message_date,media_path,source_channel,emoji_used,youtube_links
0,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15208,NEUROPRO CARE Price 5500 birr Telegram @Lobe...,2025-01-31 12:40:44+00:00,photos/@lobelia4cosmetics_15208.jpg,lobelia4cosmetics_data,👉☎🏍🏍🏍,No YouTube link
1,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15207,ENSURE 850GM Price 3800 birr Telegram https:/...,2025-01-31 12:40:44+00:00,photos/@lobelia4cosmetics_15207.jpg,lobelia4cosmetics_data,👉☎🏍🏍🏍,No YouTube link
2,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15206,NIDO 1+ 2.2KG Price 6500 birr Telegram @Lobe...,2025-01-31 12:40:44+00:00,photos/@lobelia4cosmetics_15206.jpg,lobelia4cosmetics_data,👉☎🏍🏍🏍,No YouTube link
3,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15205,Enfagrow A+ Price 5500 birr Telegram https://...,2025-01-31 12:40:44+00:00,photos/@lobelia4cosmetics_15205.jpg,lobelia4cosmetics_data,👉☎🏍🏍🏍,No YouTube link
4,Lobelia pharmacy and cosmetics,@lobelia4cosmetics,15204,go & grow 850gm Price 5500 birr Telegram http...,2025-01-31 12:40:44+00:00,photos/@lobelia4cosmetics_15204.jpg,lobelia4cosmetics_data,👉☎🏍🏍🏍,No YouTube link


2025-01-31 20:16:39,673 - INFO - ✅ Cleaned data saved successfully to 'cleaned_data.csv'.


✅ Cleaned data saved successfully to 'cleaned_data.csv'.


In [ ]:
import pandas as pd
import logging
import re
import os
import emoji
from dotenv import load_dotenv
from sqlalchemy import create_engine

# Ensure logs folder exists
os.makedirs("../logs", exist_ok=True)

# Configure logging to write to file & display in Jupyter Notebook
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    handlers=[
        logging.FileHandler("../logs/data_cleaning.log"),  # Log to file
        logging.StreamHandler()  # Log to Jupyter Notebook output
    ]
)

# Load environment variables
load_dotenv("../.env")

DB_HOST = os.getenv("DB_HOST")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
DB_PORT = os.getenv("DB_PORT")


def get_db_connection():
    """ Create and return database engine. """
    try:
        engine = create_engine(f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}")
        logging.info("✅ Database connection established successfully.")
        return engine
    except Exception as e:
        logging.error(f"❌ Error establishing database connection: {e}")
        raise

def load_csv(file_path):
    """ Load CSV file into a Pandas DataFrame. """
    try:
        df = pd.read_csv(file_path)
        logging.info(f"✅ CSV file '{file_path}' loaded successfully.")
        return df
    except Exception as e:
        logging.error(f"❌ Error loading CSV file: {e}")
        raise

def clean_text(text):
    """ Standardize text by removing newline characters and unnecessary spaces. """
    if pd.isna(text):
        return "No Message"
    return re.sub(r'\n+', ' ', text).strip()

def extract_emojis(text):
    """ Extract emojis from text, return 'No emoji' if none found. """
    emojis = ''.join(c for c in text if c in emoji.EMOJI_DATA)
    return emojis if emojis else "No emoji"

def remove_emojis(text):
    """ Remove emojis from the message text. """
    return ''.join(c for c in text if c not in emoji.EMOJI_DATA)

def extract_youtube_links(text):
    """ Extract YouTube links from text, return 'No YouTube link' if none found. """
    youtube_pattern = r"(https?://(?:www\.)?(?:youtube\.com|youtu\.be)/[^\s]+)"
    links = re.findall(youtube_pattern, text)
    return ', '.join(links) if links else "No YouTube link"

def remove_youtube_links(text):
    """ Remove YouTube links from the message text. """
    youtube_pattern = r"https?://(?:www\.)?(?:youtube\.com|youtu\.be)/[^\s]+"
    return re.sub(youtube_pattern, '', text).strip()

def clean_dataframe(df):
    """ Perform all cleaning and standardization steps while avoiding SettingWithCopyWarning. """
    try:
        df = df.drop_duplicates(subset=["ID"]).copy()  # Ensure a new copy
        logging.info("✅ Duplicates removed from dataset.")

        # ✅ Convert Date to datetime format, replacing NaT with None
        df.loc[:, 'Date'] = pd.to_datetime(df['Date'], errors='coerce')
        df.loc[:, 'Date'] = df['Date'].where(df['Date'].notna(), None)
        logging.info("✅ Date column formatted to datetime.")

        # ✅ Convert 'ID' to integer for PostgreSQL BIGINT compatibility
        df.loc[:, 'ID'] = pd.to_numeric(df['ID'], errors="coerce").fillna(0).astype(int)

        # ✅ Fill missing values
        df.loc[:, 'Message'] = df['Message'].fillna("No Message")
        df.loc[:, 'Media Path'] = df['Media Path'].fillna("No Media")
        logging.info("✅ Missing values filled.")

        # ✅ Standardize text columns
        df.loc[:, 'Channel Title'] = df['Channel Title'].str.strip()
        df.loc[:, 'Channel Username'] = df['Channel Username'].str.strip()
        df.loc[:, 'Message'] = df['Message'].apply(clean_text)
        df.loc[:, 'Media Path'] = df['Media Path'].str.strip()
        logging.info("✅ Text columns standardized.")

        # ✅ Extract emojis and store them in a new column
        df.loc[:, 'emoji_used'] = df['Message'].apply(extract_emojis)
        logging.info("✅ Emojis extracted and stored in 'emoji_used' column.")
        
        # ✅ Remove emojis from message text
        df.loc[:, 'Message'] = df['Message'].apply(remove_emojis)

        # ✅ Extract YouTube links into a separate column
        df.loc[:, 'youtube_links'] = df['Message'].apply(extract_youtube_links)
        logging.info("✅ YouTube links extracted and stored in 'youtube_links' column.")

        # ✅ Remove YouTube links from message text
        df.loc[:, 'Message'] = df['Message'].apply(remove_youtube_links)

        # ✅ Rename columns to match PostgreSQL schema
        df = df.rename(columns={
            "Channel Title": "channel_title",
            "Channel Username": "channel_username",
            "ID": "message_id",
            "Message": "message",
            "Date": "message_date",
            "Media Path": "media_path",
            "emoji_used": "emoji_used",
            "youtube_links": "youtube_links"
        })

        logging.info("✅ Data cleaning completed successfully.")
        return df
    except Exception as e:
        logging.error(f"❌ Data cleaning error: {e}")
        raise

def save_cleaned_data(df, output_path):
    """ Save cleaned data to a new CSV file. """
    try:
        df.to_csv(output_path, index=False)
        logging.info(f"✅ Cleaned data saved successfully to '{output_path}'.")
        print(f"✅ Cleaned data saved successfully to '{output_path}'.")
    except Exception as e:
        logging.error(f"❌ Error saving cleaned data: {e}")
        raise

def store_data_in_db(df, table_name, engine):
    """ Store cleaned data in the database. """
    try:
        df.to_sql(table_name, engine, if_exists='replace', index=False)
        logging.info(f"✅ Cleaned data stored successfully in table '{table_name}'.")
        print(f"✅ Cleaned data stored successfully in table '{table_name}'.")
    except Exception as e:
        logging.error(f"❌ Error storing data in database: {e}")
        raise

# Run each step interactively in Jupyter Notebook
input_csv = '../merged_data.csv'
output_csv = 'cleaned_data.csv'
table_name = 'cleaned_data'

# Load the data
df = load_csv(input_csv)

# Clean the data
cleaned_df = clean_dataframe(df)

# Save the cleaned data
save_cleaned_data(cleaned_df, output_csv)

# Store the cleaned data in the database
engine = get_db_connection()
store_data_in_db(cleaned_df, table_name, engine)

2025-02-02 19:19:38,915 - INFO - ✅ CSV file '../merged_data.csv' loaded successfully.
2025-02-02 19:19:38,934 - INFO - ✅ Duplicates removed from dataset.
2025-02-02 19:19:39,001 - INFO - ✅ Date column formatted to datetime.
2025-02-02 19:19:39,053 - INFO - ✅ Missing values filled.


DB_HOST: your_db_host
DB_NAME: your_db_name
DB_USER: your_db_user
DB_PASSWORD: your_db_password
DB_PORT: 5432  # Default PostgreSQL port


2025-02-02 19:19:39,081 - INFO - ✅ Text columns standardized.
2025-02-02 19:19:39,101 - INFO - ✅ Emojis extracted and stored in 'emoji_used' column.
2025-02-02 19:19:39,133 - INFO - ✅ YouTube links extracted and stored in 'youtube_links' column.
2025-02-02 19:19:39,143 - INFO - ✅ Data cleaning completed successfully.
2025-02-02 19:19:39,164 - INFO - ✅ Cleaned data saved successfully to 'cleaned_data.csv'.
2025-02-02 19:19:39,184 - ERROR - ❌ Error establishing database connection: invalid literal for int() with base 10: '5432  # Default PostgreSQL port'


✅ Cleaned data saved successfully to 'cleaned_data.csv'.


ValueError: invalid literal for int() with base 10: '5432  # Default PostgreSQL port'